In [1]:
# ---------------Load Data-----------------
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random as random
import os as os
%matplotlib inline 

# data = pd.read_csv('data/training.csv')
print('libraries imported!')


libraries imported!


# Def getting stock data (getting next batch)

In [2]:
stockFiles = os.listdir('stockData')
predictionsCount = 3 #the High, Low, and close over the course of the prediction period
setupArrayLength = 250 # this should be the numDays you want in a setup x 5 

def getStockData(fileName, start, setupLength, predictLength):
    setupData = []
    file = open('stockData/' + fileName)
    fLines = file.readlines()
    end = start + setupLength + predictLength -1
    predictStartDate = fLines[start + predictLength -1].split(',')[0]
    predictEndDate = fLines[start].split(',')[0]
    high = 0
    low = 1000000
    
    # get the 'old' stock data
    for i in range(end, start-1, -1):
        line = fLines[i].strip()
        lineArray = line.split(',')
        for k in range(1,7):
            lineArray[k] = float(lineArray[k])
        x = lineArray[6]/lineArray[4]

        if(i>=(start+predictLength)):
            for j in range(1,5):
                setupData.append(lineArray[j]*x)
            setupData.append(lineArray[5])

    
    # get the high/low of 'new' stock data (we try and predict this)
        if(i<(start + predictLength)):
            if (lineArray[2]*x > high):
                high = lineArray[2]*x
            if (lineArray[3]*x < low):
                low = lineArray[3]*x
                
        close = float(fLines[start].split(',')[4])*x
    
    file.close()
    
    
    # ----------------------convert data to 0=>1 instead of prices
    tmpHi = 0
    tmpLow = 1000000
    tmpVolHi = 0
    tmpVolLow = 1000000000
    
    for i in range(0,len(setupData)): # find the His and Lows for vol and price
        if(5*(i//5)+1 == i):
            if(setupData[i] > tmpHi):
                tmpHi = setupData[i]
        if(5*(i//5)+2 == i):
            if(setupData[i] < tmpLow):
                tmpLow = setupData[i]
        if(5*(i//5)+4 == i):
            if(setupData[i] > tmpVolHi):
                tmpVolHi = setupData[i]
            if(setupData[i] < tmpVolLow):
                tmpVolLow = setupData[i]
    
    for i in range(0,len(setupData)): # convert the data to values between 0 and 1
        if(5*(i//5)+4 != i):
            setupData[i] = (setupData[i] - tmpLow)/(tmpHi-tmpLow)
        if(5*(i//5)+4 == i):
            setupData[i] = (setupData[i] - tmpVolLow)/(tmpVolHi-tmpVolLow)
        
    futureLabels = [high,low,close]
    symbolAndPredictDates = [fileName.split('.')[0], predictStartDate, predictEndDate]
            
    return setupData, futureLabels, symbolAndPredictDates
 


def getBatch(setupLength, predictLength):
    randIndex = random.randint(0,950)
    fileName = stockFiles[randIndex]
    file = open('stockData/' + fileName)
    fLines = file.readlines()
    file.close()
    start = random.randint(1, (len(fLines) - setupLength - predictLength - 1))
    spySetupData, spyLabels, labelDates = getStockData('SPY.txt',start,setupLength,predictLength)
    stockSetupData, stockLabels, labelDates = getStockData(fileName,start,setupLength,predictLength)
    setupData = []
    futureLabels = []
    
    setupData.append(stockSetupData)
    setupData.append(spySetupData)
    futureLabels.append(stockLabels)
    
    return np.asarray(setupData,dtype=np.float32), np.asarray(futureLabels,dtype=np.float32), labelDates
    


print('get Data methods defined')

get Data methods defined


# Set up our CNN methods

In [3]:
# set up our weights (or kernals?) and biases for each pixel
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(.1, shape=shape, dtype=tf.float32)
    return tf.Variable(initial)

# convolution
def conv2d(x, W):
    return tf.nn.conv2d(x, W, [1,1,1,1], 'SAME')

# pooling
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')



# placeholder variables
# stock_setupData
x = tf.placeholder(tf.float32, shape=[None, setupArrayLength])

# stockPredictionData
y_ = tf.placeholder(tf.float32, shape=[None, predictionsCount])



print('methods defined')

methods defined


# set up our CNN

In [4]:
# ----------------first convolutional layer---------------------------
W_conv1 = weight_variable([1, 5, 1, 32]) # filterHeight,filterWidth,inChannels,outFeatures
b_conv1 = bias_variable([32])

# x,l = (getStockData('AXS.txt', 2,1))

# turn shape(stockDaysCount,5)  into   (?,stockDaysCount,5,1) ... so it will work with the conv2d method
stockDaysCount = int(setupArrayLength/5)
stock = tf.reshape(x, [-1,stockDaysCount,5,1])  # numStocks,stockDaysCount,width,numChannels
# print (stock.get_shape()) # =>(?,stockDaysCount,5,1)

h_conv1 = tf.nn.relu(conv2d(stock, W_conv1) + b_conv1)
# print (h_conv1.get_shape()) # => (?, stockDaysCount, 5, 32)


# ------------------------------not really sure we need max pooling?  there aren't that many data points
# h_pool1 = max_pool_2x2(h_conv1)
# print (h_pool1.get_shape()) # => (?, 48, 48, 32)




#-------------------------- second convolutional layer-----------------
W_conv2 = weight_variable([1, 5, 32, 64]) # filterHeight,filterWidth,inChannels,outFeatures
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
# print (h_conv2.get_shape()) # => (?, stockDaysCount,5,64)

#------------------again----------------no pooling
# h_pool2 = max_pool_2x2(h_conv2)
# print (h_pool2.get_shape()) # => (?, 24, 24, 64)



#-------------------------fully connected layer-----------------------
W_fc1 = weight_variable([stockDaysCount * 5 * 64, 1024])
b_fc1 = bias_variable([1024])

# (?, 24, 24, 64) => (?, 36864)
h_conv2_flat = tf.reshape(h_conv2, [-1, stockDaysCount*5*64])
# print h_pool2_flat.get_shape() = (?, 36864)

h_fc1 = tf.nn.relu(tf.matmul(h_conv2_flat, W_fc1) + b_fc1)
# print (h_fc1.get_shape()) # => (?, 1024)



# ---------------------------------readout layer-------------------------------
W_fc2 = weight_variable([1024, predictionsCount])
b_fc2 = bias_variable([predictionsCount])

readout_layer = (tf.matmul(h_fc1, W_fc2) + b_fc2)
# print (readout_layer.get_shape()) #=> (?, 30)



print('Convolutional Nueral Net Defined')

Convolutional Nueral Net Defined


# define training step and train

In [5]:
prediction_delta = tf.reduce_sum(tf.abs((readout_layer - y_)))#/y_[0:1,:]*200))
train_step = tf.train.AdamOptimizer(1e-04).minimize(prediction_delta)
# train_step = tf.train.GradientDescentOptimizer(.5).minimize(prediction_delta)


sess = tf.Session()
sess.run(tf.initialize_all_variables())
# xx, yy, zz = getBatch(50,10)
   

for i in range(0,10000):
    xx, yy, zz = getBatch(50,10)
    sess.run(train_step, feed_dict={x: xx, y_: yy})
    if(i%25 == 0):
        print(zz[0] + ' prediction_delta ' + str(i+1) + 
              ': ' + str(sess.run((prediction_delta/y_[0,0])*200, feed_dict={x: xx, y_: yy})) + '%')
#         print('prediction_dates: ' + str(zz))
#         print('corect hi/low/close: ' + str(yy))
#         print('predicted hi/low/close: ' + str(sess.run(readout_layer, feed_dict={x: xx, y_: yy})))

HDV prediction_delta 1: 1037.1%
ENR prediction_delta 26: 209.05%
BHP prediction_delta 51: 149.043%
CYBR prediction_delta 76: 210.88%
IP prediction_delta 101: 4472.24%
QEP prediction_delta 126: 1301.12%
WYN prediction_delta 151: 195.293%
PSA prediction_delta 176: 447.722%
RSPP prediction_delta 201: 120.232%
IGE prediction_delta 226: 252.361%
PAYC prediction_delta 251: 364.399%
TXRH prediction_delta 276: 316.151%
IYE prediction_delta 301: 510.44%
XME prediction_delta 326: 210.324%
SINA prediction_delta 351: 489.212%
ITC prediction_delta 376: 1269.1%
JNPR prediction_delta 401: 862.225%
DIS prediction_delta 426: 235.169%
AAXJ prediction_delta 451: 432.583%
KBR prediction_delta 476: 239.142%
GGP prediction_delta 501: 100.621%
DVA prediction_delta 526: 484.735%
TAP prediction_delta 551: 171.084%
SNE prediction_delta 576: 406.686%
WDC prediction_delta 601: 79.5576%
KATE prediction_delta 626: 1133.07%
PAA prediction_delta 651: 1593.58%
ARRS prediction_delta 676: 4770.62%
FAF prediction_delta 7

KeyboardInterrupt: 